In [1]:
import os
os.environ["NAPARI_ASYNC"] = "1"
#os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [2]:
import pandas as pd
import tensorstore as ts
import json
from copy import deepcopy
spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": "/Volumes/Extreme SSD/aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = '/Volumes/Extreme SSD/aligned_image5_image.zarr'
image = ts.open(image_spec).result()[:,0,:,:]
bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv')
display(bboxes_df.head())
with open('/Volumes/Extreme SSD/aligned_image5_labels.splits.json') as f:
    splits = json.load(f, object_hook=lambda d: {int(k): [int(_v) for _v in v] for k, v in d.items()})

,frame,label,min_y,min_x,max_y,max_x
0,0,1,13,1865,57,1909
1,0,2,19,4954,60,4994
2,0,3,29,2832,70,2876
3,0,4,35,1633,67,1669
4,0,5,38,2756,80,2797


In [3]:
import numpy as np
import tensorstore as ts
import trackarray_tensorstore as tats

print(labels.shape)
ta = tats.TrackArray(labels, splits, {}, bboxes_df)

(1965, 19991, 15437)


In [4]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()

In [ ]:
widget = StateMachineWidget(viewer, ta, image, crop_size=2048)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [    0 10769  5124]
INFO:napari_travali2._logging:Region selected: coords [    0 10769  5124]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(9745), np.int64(11793), None), slice(np.int64(4100), np.int64(6148), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(75.0), np.float64(11531.733911778325), np.float64(5568.950963560612))
DEBUG:napari_travali2._logging:data coordinates: [  75.         1786.73391178 1468.95096356]
INFO:napari_travali2._logging:clicked at [  75 1787 146